<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/4_hsv-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-08 17:49:08--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11830 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.55K  --.-KB/s    in 0s      

2022-06-08 17:49:08 (103 MB/s) - ‘utils.py’ saved [11830/11830]



In [4]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [5]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub


from utils import ModelCreator, get_train_val_ds, get_class_weight

In [6]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [7]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [8]:
train_ds, _ = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3510 files belonging to 16 classes.


In [9]:
classes, class_weight = get_class_weight()

In [10]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tf.image.rgb_to_hsv(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [11]:
hsv_creator = ModelCreator(HUB_URL, 'HSV-model')
hsv_model = hsv_creator.make_model(img_size=IMG_SIZE)

Model: "HSV-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23500352  
                                                                 
 dense (Dense)               (None, 600)               1229400   
                                                                 
 dropout (Dropout)           (None, 600)               0         
                                                                 
 dense_1 (Dense)             (None, 16)                9616      
                                                                 
Total params: 24,739,368
Trainable params: 1,239,016
Non-trainable params: 23,500,352
_________________________________________________________________


In [12]:
callbacks = hsv_creator.get_callbacks()

# ***First*** day of training

In [ ]:
history = hsv_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=25,
                        class_weight=class_weight)

Epoch 1/25
519/519 [==============================] - 3212s 6s/step - loss: 1412.3386 - auc: 0.2989 - accuracy: 0.4707
Epoch 2/25
519/519 [==============================] - 144s 276ms/step - loss: 1670.1146 - auc: 0.3392 - accuracy: 0.5201
Epoch 3/25
519/519 [==============================] - 146s 279ms/step - loss: 1190.4730 - auc: 0.3788 - accuracy: 0.5556
Epoch 4/25
519/519 [==============================] - 147s 281ms/step - loss: 1155.5914 - auc: 0.3948 - accuracy: 0.5706
Epoch 5/25
519/519 [==============================] - 147s 280ms/step - loss: 837.3010 - auc: 0.4383 - accuracy: 0.6090
Epoch 6/25
519/519 [==============================] - 147s 280ms/step - loss: 748.4167 - auc: 0.4474 - accuracy: 0.6168
Epoch 7/25
519/519 [==============================] - 147s 281ms/step - loss: 612.3502 - auc: 0.4703 - accuracy: 0.6316
Epoch 8/25
519/519 [==============================] - 147s 282ms/step - loss: 645.6837 - auc: 0.4703 - accuracy: 0.6336
Epoch 9/25
519/519 [==================

In [ ]:
hsv_model.save('./drive/MyDrive/checkpoints/hsv-model-25.h5')

In [ ]:
history = hsv_model.fit(train_ds,
                          callbacks=callbacks,
                          epochs=35,
                          class_weight=class_weight,
                          initial_epoch=25)

Epoch 26/35
519/519 [==============================] - 148s 282ms/step - loss: 43.2054 - auc: 0.7388 - accuracy: 0.7797
Epoch 27/35
519/519 [==============================] - 149s 284ms/step - loss: 38.6738 - auc: 0.7499 - accuracy: 0.7896
Epoch 28/35
519/519 [==============================] - 148s 282ms/step - loss: 38.9391 - auc: 0.7513 - accuracy: 0.7873
Epoch 29/35
519/519 [==============================] - 147s 281ms/step - loss: 31.5158 - auc: 0.7726 - accuracy: 0.7966
Epoch 30/35
519/519 [==============================] - 149s 282ms/step - loss: 27.9170 - auc: 0.7879 - accuracy: 0.8028
Epoch 31/35
519/519 [==============================] - 148s 282ms/step - loss: 24.6731 - auc: 0.7985 - accuracy: 0.8064
Epoch 32/35
519/519 [==============================] - 147s 282ms/step - loss: 23.6671 - auc: 0.8047 - accuracy: 0.8061
Epoch 33/35
519/519 [==============================] - 148s 282ms/step - loss: 21.8818 - auc: 0.8179 - accuracy: 0.8168
Epoch 34/35
519/519 [===================

In [ ]:
hsv_model.save('./drive/MyDrive/checkpoints/hsv-model-35.h5')

# ***Second*** day

In [13]:
hsv_model = tf.keras.models.load_model('./drive/MyDrive/checkpoints/hsv-model-35.h5',
                                       custom_objects={'KerasLayer':hub.KerasLayer})

In [14]:
history = hsv_model.fit(train_ds,
                          callbacks=callbacks,
                          epochs=55,
                          class_weight=class_weight,
                          initial_epoch=35)

Epoch 36/55
519/519 [==============================] - 2125s 4s/step - loss: 16.8194 - auc: 0.8420 - accuracy: 0.8271
Epoch 37/55
519/519 [==============================] - 146s 279ms/step - loss: 14.6472 - auc: 0.8539 - accuracy: 0.8346
Epoch 38/55
519/519 [==============================] - 147s 281ms/step - loss: 13.9205 - auc: 0.8621 - accuracy: 0.8383
Epoch 39/55
519/519 [==============================] - 147s 281ms/step - loss: 13.1162 - auc: 0.8685 - accuracy: 0.8420
Epoch 40/55
519/519 [==============================] - 147s 282ms/step - loss: 12.4649 - auc: 0.8695 - accuracy: 0.8414
Epoch 41/55
519/519 [==============================] - 147s 281ms/step - loss: 11.7970 - auc: 0.8735 - accuracy: 0.8439
Epoch 42/55
519/519 [==============================] - 147s 281ms/step - loss: 11.4112 - auc: 0.8808 - accuracy: 0.8476
Epoch 43/55
519/519 [==============================] - 148s 282ms/step - loss: 11.0059 - auc: 0.8865 - accuracy: 0.8497
Epoch 44/55
519/519 [=====================

In [15]:
hsv_model.save('drive/MyDrive/checkpoints/hsv-model-55.h5')

In [16]:
history = hsv_model.fit(train_ds,
                          callbacks=callbacks,
                          epochs=65,
                          class_weight=class_weight,
                          initial_epoch=55)

Epoch 56/65
519/519 [==============================] - 148s 282ms/step - loss: 7.5150 - auc: 0.9172 - accuracy: 0.8707
Epoch 57/65
519/519 [==============================] - 147s 281ms/step - loss: 7.5869 - auc: 0.9174 - accuracy: 0.8706
Epoch 58/65
519/519 [==============================] - 147s 281ms/step - loss: 7.5320 - auc: 0.9169 - accuracy: 0.8729
Epoch 59/65
519/519 [==============================] - 148s 282ms/step - loss: 7.4869 - auc: 0.9186 - accuracy: 0.8711
Epoch 60/65
519/519 [==============================] - 147s 281ms/step - loss: 7.2472 - auc: 0.9200 - accuracy: 0.8754
Epoch 61/65
519/519 [==============================] - 148s 282ms/step - loss: 7.2228 - auc: 0.9208 - accuracy: 0.8737
Epoch 62/65
519/519 [==============================] - 148s 282ms/step - loss: 7.1548 - auc: 0.9226 - accuracy: 0.8770
Epoch 63/65
519/519 [==============================] - 147s 281ms/step - loss: 7.0651 - auc: 0.9229 - accuracy: 0.8762
Epoch 64/65
519/519 [===========================

In [17]:
hsv_model.save('drive/MyDrive/checkpoints/hsv-model-65.h5')

In [18]:
history = hsv_model.fit(train_ds,
                          callbacks=callbacks,
                          epochs=75,
                          class_weight=class_weight,
                          initial_epoch=65)

Epoch 66/75
519/519 [==============================] - 147s 281ms/step - loss: 7.0097 - auc: 0.9233 - accuracy: 0.8771
Epoch 67/75
519/519 [==============================] - 147s 281ms/step - loss: 6.9015 - auc: 0.9274 - accuracy: 0.8763
Epoch 68/75
519/519 [==============================] - 147s 281ms/step - loss: 6.8915 - auc: 0.9269 - accuracy: 0.8769
Epoch 69/75
519/519 [==============================] - 147s 281ms/step - loss: 6.9313 - auc: 0.9231 - accuracy: 0.8786
Epoch 70/75
519/519 [==============================] - 147s 280ms/step - loss: 6.8377 - auc: 0.9269 - accuracy: 0.8779
Epoch 70: early stopping


In [20]:
hsv_model.save('drive/MyDrive/checkpoints/hsv-model-67.h5')